In [54]:
import mysql.connector
import stuff

In [55]:
cnx = mysql.connector.connect(
    host = stuff.host,
    user = stuff.user,
    passwd = stuff.password)

In [56]:
cursor = cnx.cursor()

In [57]:
cnx.database = 'movie_project'

In [58]:
import requests
from bs4 import BeautifulSoup as BS
import pandas as pd

In [59]:
#make sure to put quotes around your url
def request_func(url):
    user_agent = {'User-agent': 'Mozilla/5.0'}
    page = requests.get(url, headers = user_agent)
    soup = BS(page.content, 'html.parser')
    return soup

In [60]:
#Extracting the critic scores
def critic_func (soup):
    critic_scores=soup.find_all('div', class_='metascore_w large movie positive')
    i=0
    critic_scores_score=[]
    critic_scores_duplicate=[]
    for thing in critic_scores:
        if i%2==0:
            critic_scores_score.append(critic_scores[i].string)
        elif i%2!=0:
            critic_scores_duplicate.append(critic_scores[i].string)
        i+=1
        
    critic_scores_metacritic=[]
    for item in critic_scores_score:
        try:
            maybe=float(item)
            critic_scores_metacritic.append(maybe)
        except:
            critic_scores_metacritic.append(None)

    return critic_scores_metacritic

In [61]:
#Extracting the critic scores
#had to make another funciton for when the color of the box in the critic scores turn yellow
def critic_yellow_func (soup):
    critic_scores=soup.find_all('div', class_='metascore_w large movie mixed')
    i=0
    critic_scores_score=[]
    critic_scores_duplicate=[]
    for thing in critic_scores:
        if i%2==0:
            critic_scores_score.append(critic_scores[i].string)
        elif i%2!=0:
            critic_scores_duplicate.append(critic_scores[i].string)
        i+=1
        
    critic_scores_metacritic_yellow=[]
    for item in critic_scores_score:
        try:
            maybe=float(item)
            critic_scores_metacritic_yellow.append(maybe)
        except:
            critic_scores_metacritic_yellow.append(None)

    return critic_scores_metacritic_yellow

In [62]:
#Extracting the critic scores
#had to make another funciton for when the color of the box in the critic scores turn red
def critic_red_func (soup):
    critic_scores=soup.find_all('div', class_='metascore_w large movie negative')
    i=0
    critic_scores_score=[]
    critic_scores_duplicate=[]
    for thing in critic_scores:
        if i%2==0:
            critic_scores_score.append(critic_scores[i].string)
        elif i%2!=0:
            critic_scores_duplicate.append(critic_scores[i].string)
        i+=1
        
    critic_scores_metacritic_red=[]
    for item in critic_scores_score:
        try:
            maybe=float(item)
            critic_scores_metacritic_red.append(maybe)
        except:
            critic_scores_metacritic_red.append(None)

    return critic_scores_metacritic_red

In [63]:
#Extract user scores
def user_func (soup):
    user_score = soup.find_all('a', class_= "metascore_anchor")
    user_score_scores=[]
    for thing in user_score:
        user_score_scores.append(thing.select('div')[0].string)
    usr_score=user_score_scores[2:301:3]
    usr_score2=[]
    for item in usr_score:
        try:
            maybe=float(item)
            usr_score2.append(maybe)
        except:
            usr_score2.append(None)
    return usr_score2


In [64]:
#Extract movie titles
def title_func (soup):
    title = soup.find_all('a', class_= "title")
    titles=[]
    for thing in title:
        titles.append(thing.select('h3')[0].string)
    return titles

In [65]:
#making tuple for all data to insert into sql
def create_tuple_func (critic_scores_metacritic,usr_score2,titles):
    i=0
    All_data_metacritic=[]
    for thing in range(len(titles)):
        new_tuple = (titles[i],usr_score2[i],critic_scores_metacritic[i])
        All_data_metacritic.append(new_tuple)
        i+=1
    return All_data_metacritic

In [ ]:
#inserts data into sql table
def insert_func (cnx, cursor, All_data_metacritic):
    insert_stm = """INSERT INTO mojo_data (title, days_in_theater, opening_revenue, gross_revenue, days_in_theatre_with_none) 
    VALUES (%s, %s, %s, %s)"""
    cursor.executemany(insert_stm, All_data_mojo)
    cnx.commit()

In [66]:
def process(url):
    soup=request_func(url)
    critic_scores_metacritic=critic_func(soup)
    critic_scores_metacritic_yellow=critic_yellow_func(soup)
    critic_scores_metacritic_red=critic_red_func(soup)
    critic_scores_metacritic=critic_scores_metacritic+critic_scores_metacritic_yellow+critic_scores_metacritic_red
    usr_score2=user_func(soup)
    titles=title_func(soup)
    All_data_metacritic = create_tuple_func(critic_scores_metacritic,usr_score2,titles)
    insert_func(cnx, cursor, All_data_metacritic)